<a href="https://colab.research.google.com/github/ueki5/colaboratory/blob/main/mathbullet_RAG_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- このノートブックの内容は2024年5月時点の [LangChain 公式ドキュメント](https://python.langchain.com/v0.1/docs/get_started/introduction)に依拠しています。
- 事前に社内規定PDFを概要欄リンクからダウンロードし、動画内の説明に従ってアップロードしておいてください。

## 準備：API キーの設定

In [ ]:
!pip install -q openai==1.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


In [ ]:

from google.colab import userdata
import openai

# OPENAI_API_KEY を取得
openai.api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = openai.api_key

# RAG（LLM の知識拡張）
RAG を活用することで、LLM は学習データに含まれない知識を得ることができます。

参照用データとしては、現在 LangChain は以下に対応しています：
- .txt
- .csv
- .html
- .json
- .md
- .pdf
- MS Office ファイル

今回は PDF からの知識抽出を試してみましょう。

In [ ]:
!pip install -q pypdf
!pip install -q langchain==0.1.20
!pip install -q langchain-openai==0.1.6
!pip install -q langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.3 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [ ]:
loader = PyPDFDirectoryLoader("/content/ref")

In [ ]:
data = loader.load()

In [ ]:
data

[Document(page_content='⼈事規定\n1⼈事規定\n\ue072\ue094\x00採⽤\n1. 1. 採⽤は、必要に応じて公正な選考を経て⾏う。\n1.2. 新⼊社員には、会社の理念および業務内容に関する研修を実施する。\n1. 3 . 採⽤にあたり、適性検査および⾯接を実施する。\n\ue073\ue094\x00昇進‧昇給\n2. 1. 昇進および昇給は、業績および勤務態度に基づき、公正に⾏う。\n2.2. 昇進および昇給の基準は、年に⼀度⾒直すものとする。\n2. 3 . 昇進候補者は、上司の推薦および⼈事評価を基に決定する。\n\ue074\ue094\x00退職‧解雇\n3 . 1. 社員が⾃⼰都合で退職する場合、退職希望⽇の 1 か⽉前までに申し出ること。\n3 .2. 解雇は、正当な理由がある場合に限り、事前に通知した上で⾏う。\n3 . 3 . 退職時には、会社の財産および機密情報を適切に返却しなければならない。', metadata={'source': '/content/ref/人事規定.pdf', 'page': 0}),
 Document(page_content='休暇‧休業規定\n1休暇‧休業規定\n\ue072\ue094\x00年次有給休暇\n1. 1. 社員は勤務開始⽇から 6 か⽉経過後、 10 ⽇の有給休暇を取得できる。\n1.2. 年次有給休暇は翌年度に繰り越すことができるが、最⼤ 20 ⽇までとする。\n1. 3 . 有給休暇の取得は、業務に⽀障がない範囲で事前に申請し、承認を得ること。\n\ue073\ue094\x00特別休暇\n2. 1. 結婚、出産、忌引などの特別な事情がある場合、特別休暇を付与する。\n2.2. 特別休暇の⽇数および取得⽅法は、個別の事情に応じて決定する。\n2. 3 . 特別休暇の取得は、事前に⼈事部に申請し、承認を得ること。\n\ue074\ue094\x00産前産後休業\n3 . 1. 妊娠中の⼥性社員は、産前 6 週間および産後 8 週間の休業を取得できる。\n3 .2. 産前産後休業中は、給与の⽀給を停⽌するが、社会保険料は会社が負担する。\n3 . 3 . 産前産後休業の取得は、医師の診断書を添付し、事前に⼈事部に申請するこ\nと。\n\ue075\ue094

In [ ]:
len(data)

6

## 文埋め込みと類似文書検索

In [ ]:
from langchain_openai import OpenAIEmbeddings

In [ ]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
from langchain_chroma import Chroma

In [ ]:
# ベクトルデータベースを作る
db = Chroma.from_documents(data, embeddings_model)

In [ ]:
# 抽出器を作る
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":1})

In [ ]:
query = "社員の倫理的責任"

retriever.invoke(query)

[Document(page_content='倫理規範\n1倫理規範\n\ue072\ue094\x00倫理とコンプライアンス\n1. 1. 社員は、⾼い倫理観を持ち、公正に業務を遂⾏すること。\n1.2. コンプライアンスを徹底し、法令違反や不正⾏為を⾏わないこと。\n1. 3 . 会社の利益を最優先に考え、利益相反⾏為を避けること。\n\ue073\ue094\x00ハラスメント禁⽌\n2. 1. 社員間のハラスメントを禁⽌し、健全な職場環境を維持すること。\n2.2. ハラスメントの疑いがある場合は、速やかに⼈事部に報告すること。\n2. 3 . ハラスメントに対する調査および対策を迅速に⾏うこと。\n\ue074\ue094\x00社会貢献\n3 . 1. 社員は、地域社会や環境への貢献を意識し、社会的責任を果たすこと。\n3 .2. ボランティア活動や環境保護活動への参加を奨励する。\n3 . 3 . 社会貢献活動に対する⽀援を会社として⾏う。', metadata={'page': 0, 'source': '/content/ref/倫理規範.pdf'})]

## Retrieval を活用したチャットモデルの制御

### まずはただ聞いてみる

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
gpt4o = ChatOpenAI(model="gpt-4o", temperature=1.0)
out = gpt4o.invoke("賞与は年何回支給されますか？")
print(out.content)

賞与（ボーナス）の支給回数は会社や業界によって異なりますが、一般的には年に2回支給されることが多いです。通常、夏と冬に支給されることが一般的です。具体的な支給月は企業ごとに異なり、就業規則や労働契約書に記載されていることが多いです。

一部の企業では、業績によって春や秋に追加の賞与が支給されることもあります。賞与の支給回数や金額に関する詳細は、企業の人事部や上司に確認するのが確実です。


### 準備：プロンプトテンプレートを作る

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="以下を参照して、質問に答えてください。\n\n{context}\n\n質問：{question}"
)

In [ ]:
example = {"question":"これは質問です", "context":"これは外部知識です"}

prompt_template.invoke(example)

StringPromptValue(text='以下を参照して、質問に答えてください。\n\nこれは外部知識です\n\n質問：これは質問です')

### 準備：抽出器を作る

In [ ]:
# ベクトルデータベースを作る
db = Chroma.from_documents(data, embeddings_model)

# 抽出器を作る
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [ ]:
retriever.invoke("賞与は年何回支給されますか？")

[Document(page_content='給与規定\n1給与規定\n\ue072\ue094\x00給与の⽀給\n1. 1. 給与は、毎⽉ 25 ⽇に銀⾏振込により⽀給する。\n1.2. ⽀給⽇が休⽇にあたる場合は、その前営業⽇に⽀給する。\n1. 3 . 給与明細は、給与⽀給⽇に電⼦メールで配布する。\n\ue073\ue094\x00賞与\n2. 1. 賞与は、業績および個⼈の勤務成績に基づき、年 2 回⽀給する。\n2.2. 賞与の⽀給額および⽀給⽇は、毎年⾒直すものとする。\n2. 3 . 賞与⽀給の基準は、会社の業績⽬標達成度および個⼈の評価による。\n\ue074\ue094\x00経費精算\n3 . 1. 会社業務に必要な経費は、事前に承認を得た上で精算する。\n3 .2. 経費精算は、毎⽉末に締め、翌⽉初に⽀給する。\n3 . 3 . 経費精算の際は、領収書および証憑を提出すること。', metadata={'page': 0, 'source': '/content/ref/給与規定.pdf'})]

In [ ]:
retrieved = retriever.invoke("賞与は年何回支給されますか？")

print(retrieved[0].page_content)

給与規定
1給与規定
 給与の⽀給
1. 1. 給与は、毎⽉ 25 ⽇に銀⾏振込により⽀給する。
1.2. ⽀給⽇が休⽇にあたる場合は、その前営業⽇に⽀給する。
1. 3 . 給与明細は、給与⽀給⽇に電⼦メールで配布する。
 賞与
2. 1. 賞与は、業績および個⼈の勤務成績に基づき、年 2 回⽀給する。
2.2. 賞与の⽀給額および⽀給⽇は、毎年⾒直すものとする。
2. 3 . 賞与⽀給の基準は、会社の業績⽬標達成度および個⼈の評価による。
 経費精算
3 . 1. 会社業務に必要な経費は、事前に承認を得た上で精算する。
3 .2. 経費精算は、毎⽉末に締め、翌⽉初に⽀給する。
3 . 3 . 経費精算の際は、領収書および証憑を提出すること。


### 準備：フォーマッターを作る
抽出器の出力を生テキストの形に成形する関数を用意しておく。

In [ ]:
def text_formatter(retriever_output):
  raw_text = retriever_output[0].page_content
  raw_text_wo_newline = raw_text.replace("\n", "")
  return raw_text_wo_newline

In [ ]:
text_formatter(retrieved)

'給与規定1給与規定\ue072\ue094\x00給与の⽀給1. 1. 給与は、毎⽉ 25 ⽇に銀⾏振込により⽀給する。1.2. ⽀給⽇が休⽇にあたる場合は、その前営業⽇に⽀給する。1. 3 . 給与明細は、給与⽀給⽇に電⼦メールで配布する。\ue073\ue094\x00賞与2. 1. 賞与は、業績および個⼈の勤務成績に基づき、年 2 回⽀給する。2.2. 賞与の⽀給額および⽀給⽇は、毎年⾒直すものとする。2. 3 . 賞与⽀給の基準は、会社の業績⽬標達成度および個⼈の評価による。\ue074\ue094\x00経費精算3 . 1. 会社業務に必要な経費は、事前に承認を得た上で精算する。3 .2. 経費精算は、毎⽉末に締め、翌⽉初に⽀給する。3 . 3 . 経費精算の際は、領収書および証憑を提出すること。'

### 抽出・補完・生成のチェインを作る

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
llm = ChatOpenAI(model="gpt-4o")

retriever、prompt_template、llmが用意できたので、これらを組み合わせて、情報の流れを定義する。LangChainでは、これから書いていくような「インプットからアウトプットまでの流れ」をチェイン（chain）と呼ぶ。

In [ ]:

chain = ({"question":RunnablePassthrough(), "context":retriever|text_formatter}
         | prompt_template
         | llm
         )

In [ ]:
chain.invoke("賞与は年何回支給されますか？")

AIMessage(content='賞与は年に2回支給されます。', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 377, 'total_tokens': 389}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-f6310c80-3f27-47e6-a6a4-5d3e5d557471-0')

In [ ]:
chain.invoke("試用期間は何日間ですか？")

AIMessage(content='ご質問の休暇・休業規定には、試用期間に関する記載がありません。そのため、この文書からは試用期間が何日間かを判断することはできません。試用期間についての情報は、他の規定や社員ハンドブック、または会社の人事部に問い合わせることをお勧めします。', response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 543, 'total_tokens': 625}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-54ce0855-571c-4f16-84d4-621402aea078-0')